In [2]:
# Import modules: xrray, schemaorg
import xarray
from schemaorg.main import Schema 
from schemaorg.main.parse import RecipeParser                # For schema validation
from schemaorg.templates.google import make_dataset          # To generate html


# Load data with metadata (Type: netcdf)
data = xarray.open_dataset('data/ampere/20130101.1300.3600.600.north.grd.ncdf')
data

In [ ]:
## Create a 'Dataset' Schema object
SchemaObj = Schema("Dataset")

## Add a single property
SchemaObj.add_property("name","AMPERE Data For 01/01/2013 13:00:00")